## Entrenamiento de Modelo

### Relevancia de la Estrategia de MLOps

La implementación de una estrategia de MLOps en el análisis del dataset de hongos cobra una importancia crucial, especialmente cuando abordamos la pregunta: *"¿podemos determinar si un hongo es comestible basándonos en sus características físicas?"*. Esta pregunta no solo plantea un desafío analítico y de modelado significativo, sino que también implica una profunda responsabilidad ética y de seguridad. En ámbitos donde la salud humana podría estar en juego, la precisión y fiabilidad de las predicciones de nuestro modelo no son meramente objetivos deseables, sino imperativos críticos.

En este contexto, una estrategia de MLOps es esencial, ya que permite la iteración continua y sistemática sobre el modelo, buscando optimizar su desempeño mientras se asegura la precisión y fiabilidad del sistema. La incorporación de MLOps facilita una mejora constante mediante la automatización, la integración y la entrega continuas, junto con el monitoreo y mantenimiento en producción, lo que es crucial para manejar la delicada naturaleza de la pregunta de investigación y garantizar la seguridad de las predicciones.

En consonancia con este enfoque iterativo y basado en la necesidad de equilibrar la precisión con la responsabilidad, elegimos comenzar nuestro proceso de modelado con un modelo base sencillo: la regresión logística. Esta elección se fundamenta en su interpretabilidad, simplicidad y eficacia probada como punto de partida en problemas de clasificación. A partir de este modelo base, podemos evaluar su desempeño como línea de base y, apoyados por la infraestructura y prácticas que MLOps facilita, proceder a experimentar y mejorar iterativamente. Esto nos permite explorar modelos más complejos y ajustar parámetros con el objetivo final de optimizar la precisión y la seguridad de nuestras predicciones, garantizando así que nuestro sistema de predicción evolucione de manera responsable y efectiva para proteger la salud y el bienestar de las personas.

### Importación de Librerías

In [44]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
import scipy.stats as ss
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from rich import print
from rich.console import Console
from rich.table import Table

%matplotlib inline

### Carga del Dataset

In [45]:
parent_directory = os.path.dirname(os.getcwd())
path_primary_data = os.path.join(parent_directory, "data", "secondary_data.csv")
df = pd.read_csv(path_primary_data, sep=";")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61069 entries, 0 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 61069 non-null  object 
 1   cap-diameter          61069 non-null  float64
 2   cap-shape             61069 non-null  object 
 3   cap-surface           46949 non-null  object 
 4   cap-color             61069 non-null  object 
 5   does-bruise-or-bleed  61069 non-null  object 
 6   gill-attachment       51185 non-null  object 
 7   gill-spacing          36006 non-null  object 
 8   gill-color            61069 non-null  object 
 9   stem-height           61069 non-null  float64
 10  stem-width            61069 non-null  float64
 11  stem-root             9531 non-null   object 
 12  stem-surface          22945 non-null  object 
 13  stem-color            61069 non-null  object 
 14  veil-type             3177 non-null   object 
 15  veil-color         

### Limpieza de Datos

- Tratar los valores faltantes (imputación, eliminación de filas/columnas, etc.).
- Identificar y manejar valores atípicos (outliers).
- Manejar datos duplicados y/o inconsistentes.

#### Eliminación de outliers

In [46]:
def detect_outliers_iqr(dataframe):
    """
    Detecta outliers en todas las columnas numéricas de un DataFrame usando el método del rango intercuartílico (IQR).
    
    Parámetros:
    - dataframe: DataFrame de pandas que contiene las variables numéricas.
    
    Retorna:
    - Un DataFrame que contiene solo las filas que son consideradas outliers en alguna de las columnas numéricas.
    """
    outliers_df = pd.DataFrame(columns=dataframe.columns)
    
    # Selecciona solo las columnas numéricas; en caso de que se haya ingresado un DataFrame con columnas categóricas
    numeric_cols = dataframe.select_dtypes(include=['int64', 'float64'])
    
    for column in numeric_cols:
        Q1 = dataframe[column].quantile(0.25)
        Q3 = dataframe[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        # Filtra los outliers
        filter_outliers = (dataframe[column] < lower_bound) | (dataframe[column] > upper_bound)
        outliers_in_column = dataframe[filter_outliers]
        
        # Agrega los outliers al DataFrame de outliers
        outliers_df = pd.concat([outliers_df, outliers_in_column], axis=0).drop_duplicates().reset_index(drop=True)
    
    return outliers_df


In [47]:
# %%capture --no-display
# Detectando outliers en el DataFrame de variables numéricas
outliers_df = detect_outliers_iqr(df)

In [48]:
# Eliminar filas con outliers del conjunto de datos original
df_no_outliers = df.drop(outliers_df.index)
df_no_outliers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55729 entries, 5340 to 61068
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   class                 55729 non-null  object 
 1   cap-diameter          55729 non-null  float64
 2   cap-shape             55729 non-null  object 
 3   cap-surface           43419 non-null  object 
 4   cap-color             55729 non-null  object 
 5   does-bruise-or-bleed  55729 non-null  object 
 6   gill-attachment       47257 non-null  object 
 7   gill-spacing          33843 non-null  object 
 8   gill-color            55729 non-null  object 
 9   stem-height           55729 non-null  float64
 10  stem-width            55729 non-null  float64
 11  stem-root             8074 non-null   object 
 12  stem-surface          20474 non-null  object 
 13  stem-color            55729 non-null  object 
 14  veil-type             353 non-null    object 
 15  veil-color      

#### Eliminación de Variables No Deseadas

In [49]:
# Eliminar columnas con alto índice de registros nulos
df_clean = df_no_outliers.drop(columns=['veil-type', 'spore-print-color', 'veil-color', 'stem-root', 'stem-surface', 'gill-spacing'])
df_clean.head(10)

,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-color,stem-height,stem-width,stem-color,has-ring,ring-type,habitat,season
5340,e,9.58,f,NaN,g,f,s,g,7.46,21.69,u,f,f,g,w
5341,e,8.19,f,NaN,n,f,s,g,6.77,20.02,u,f,f,g,w
5342,e,9.04,f,NaN,g,f,s,g,7.00,21.65,u,f,f,g,w
5343,e,9.49,x,NaN,n,f,s,g,7.77,19.68,u,f,f,g,w
5344,e,10.01,x,NaN,n,f,s,g,7.62,21.06,u,f,f,g,a
5345,e,7.63,f,NaN,n,f,s,p,6.35,20.53,u,f,f,g,a
5346,e,7.80,f,NaN,n,f,s,p,5.52,18.09,u,f,f,d,a
5347,e,9.11,f,NaN,g,f,s,p,7.71,18.91,u,f,f,g,a
5348,e,6.72,f,NaN,n,f,s,p,6.90,19.54,u,f,f,g,a
5349,e,9.69,f,NaN,n,f,s,g,5.47,21.30,u,f,f,d,a


### División de Datos

#### Conjunto de Entrenamiento y Prueba

In [50]:
# División de datos en variables independientes (X) y dependientes (y)
X = df_clean.drop('class', axis=1)  # Características o variables independientes
y = df_clean['class']  # Variable objetivo o dependiente

# División estratificada en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Imprimir información sobre la división de datos
print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento:
(44583, 14)

Tamaño del conjunto de prueba:
(11146, 14)

#### Conjunto de Entrenamiento: Variables Numéricas y Categóricas

In [51]:
# Identificar características numéricas y categóricas en el conjunto de entrenamiento
numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X_train.select_dtypes(include=['object']).columns

### Transformación de Datos

In [52]:
# Definir transformadores para variables numéricas y categóricas
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Imputación utilizando la mediana, debido a la distribución sesgada de los datos
    ('standard_scaler', StandardScaler()),  # Estandarización
    ('min_max_scaler', MinMaxScaler())  # Normalización
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Imputación utilizando la moda
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Codificación utilizando OneHotEncoder, solo tenemos variables categóricas nominales
])

In [53]:
# Crear el preprocesador con ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('numeric', numeric_transformer, numeric_features),
        ('categorical', categorical_transformer, categorical_features)
    ])

In [54]:
# Crear el pipeline final con el preprocesador y el modelo de regresión logística
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression())  # Modelo de regresión logística
])

### Entrenamiento del Modelo

In [55]:
# Entrenar el modelo utilizando el conjunto de entrenamiento
model = pipeline.fit(X_train, y_train)

### Evaluación del Modelo

In [56]:
# Realizar predicciones utilizando el conjunto de prueba
y_pred = model.predict(X_test)

In [57]:
print(y_pred)
y_pred.shape

['p' 'e' 'e' ... 'e' 'p' 'e']

(11146,)

In [58]:
# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

# Imprimir los resultados
print("Accuracy:", accuracy)
print("\nConfusion Matrix:\n", conf_matrix)
print("\nClassification Report:\n", class_report)

Accuracy: 0.802888928763682

Confusion Matrix:
 [[3784 1078]
 [1119 5165]]

Classification Report:
               precision    recall  f1-score   support

           e       0.77      0.78      0.78      4862
           p       0.83      0.82      0.82      6284

    accuracy                           0.80     11146
   macro avg       0.80      0.80      0.80     11146
weighted avg       0.80      0.80      0.80     11146

### Evaluación del Modelo

### Validación Cruzada y Ajuste de Hiperparámetros

### Interpretación del Modelo